In [89]:
# Из списка с городами убираем дубликаты и добавляем нужную геоинфу 
import pandas as pd
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\анализ юг.xlsx',
    usecols=[2])
new_df = pd.DataFrame({'city':df.город.unique()})

In [90]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='Yandex')
new_df['full_name'] = new_df['city'].apply(lambda x: (geolocator.geocode(x,timeout=10000)))


In [91]:
new_df['full_name'] = new_df['full_name'].fillna(0)

In [92]:
new_df['latitude'] = new_df['full_name'].apply(lambda x: x.latitude if x!= 0 else 0 )

In [93]:
new_df['longitude'] = new_df['full_name'].apply(lambda x: x.longitude if x!= 0 else 0 )

In [94]:
new_df['coord'] = new_df['full_name'].apply(lambda x: (x.latitude, x.longitude))

In [98]:
from geopy.distance import geodesic
vlad = (43.024593, 44.68211)
nal = (43.4883695, 43.610560193588)
new_df['distance vlad'] = new_df['coord'].apply(lambda x: geodesic(vlad,x).km)
new_df['distance nal'] = new_df['coord'].apply(lambda x: geodesic(nal,x).km)

In [99]:
new_df.to_excel( r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\для анализ юг1.xlsx')

In [ ]:
df['coord'] = df['address'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [107]:
с = (49.65690635, 41.6993391207763)
print(geodesic(nal,(с)).km)
print(geodesic(vlad,(с)).km)

701.1226097859368
772.0011342236569


In [104]:
from geopy.geocoders import Yandex
geolocator = Yandex()
location = geolocator.geocode("Железногорск")
print(location.latitude, location.longitude)


52.337971 35.351743


In [14]:
geolocator.geocode("Астрахань,Чалбаяна/Ногина, 26/9").latitude

46.346874

In [ ]:
from geopy.geocoders import Yandex
geolocator = Yandex()
location = "Железногорск")
print(location.latitude, location.longitude)


In [120]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode("Озерск, Дзержинского")
print(location)

Уральские кабельные сети - Озерск, 35А, улица Дзержинского, Озёрск, Озёрский городской округ, Челябинская область, Уральский федеральный округ, 456785, РФ


In [ ]:
#проставляет к городу полный адрес
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application",country_bias='Ru')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
geo_df['full_address'] = geo_df['city'].apply(geocode)

In [23]:
#К адресам магазов добавляет широту и долготу
import pandas as pd
from geopy.geocoders import Yandex
geo_df = pd.read_excel(r'C:\Остатки\маги.xlsx', skiprows=11,skipfooter=1)
geo_df.columns = ['Склад', 'Название', 'Прибыль', 'Продажи', 'Норма']
geo_df = geo_df[geo_df['Норма'] > 0]
geolocator = Yandex(api_key='6e812556-1ed1-445c-9efc-727060bf204a', timeout=10000, lang="ru_RU")

# добавлено условие чтобы запрос не останавливался, если не удается найти адрес
geo_df['Latitude'] = geo_df['Название'].apply(
    lambda x: geolocator.geocode(x).latitude if geolocator.geocode(x) != None else 'not found')

geo_df['Longitude'] = geo_df['Название'].apply(
    lambda x: geolocator.geocode(x).longitude if geolocator.geocode(x) != None else 'not found')

In [29]:
geo_df.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\маги_с_координатами.xlsx')

In [99]:
geo_df['Прибыль'] = geo_df['Прибыль'].astype('int')

In [95]:
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl()

In [100]:
map_1.add_data(data=geo_df[geo_df['Latitude'] != 'not found'], name='far')

In [103]:
map_1.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'iww1b1zg4',
     'type': 'point',
     'config': {'dataId': 'far',
      'label': 'Point',
      'color': [18, 147, 154],
      'columns': {'lat': 'Latitude', 'lng': 'Longitude', 'altitude': None},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Uber Viz Diverging 3.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C',
         '#2FA7AE',
         '#5DBABF',
         '#8CCED1',
         '#BAE1E2',
         '#F8C0AA',
         '#EB9C80',
         '#DD7755',
         '#D0532B',
         '#C22E00']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
        

In [105]:
config = map_1.config
map_1.save_to_html(data={'far': geo_df[geo_df['Latitude'] != 'not found']}, config=config, file_name='продажи.html')

Map saved to продажи.html!


In [85]:
my_str = 'Кольцово раб. поселок , 19, ТЦ'
if x in my_str.isdigit():
    

IndentationError: expected an indented block (<ipython-input-92-d31224bfb593>, line 6)

In [52]:
filter(str.isdigit, "lol123")

In [27]:
geo_df['Название'].iloc[:10].apply
(lambda x: geolocator.geocode(x).latitude if geolocator.geocode(x) != None else 'not found')

0    46.346874
1    50.601195
2    45.048405
3    60.711394
4    51.500983
5    43.482368
6    45.013893
7    44.897627
8    55.438578
9    54.938943
Name: Название, dtype: float64

In [36]:
point = geolocator.geocode('Кольцово раб. поселок , 19')
print(point.latitude)
print(point.longitude)


56.424362
44.193276


In [10]:
geo_df

,Склад,Название,Прибыль,Продажи,Норма,full_address_yandex
0,10,"Астрахань,Чалбаяна/Ногина, 26/9",2.868924e+06,800,3918,"(улица Ногина, Астрахань, Россия, (46.346874, ..."
1,101,"Белгород, Богдана Хмельницкого, 103",9.113728e+06,2273,8721,"(проспект Богдана Хмельницкого, 103, Белгород,..."
2,1066,"Ставрополь, Карла Маркса, 62",2.777218e+06,807,2454,"(проспект Карла Маркса, 62, Ставрополь, Россия..."
3,1067,"Выборг, Северная, 10",7.215603e+06,1809,7799,"(Северная улица, 10, Выборг, Ленинградская обл..."
4,107,"Энгельс, Свободы, 3а",5.074621e+06,1446,5223,"(площадь Свободы, 3А, Энгельс, Саратовская обл..."
5,1074,"Нальчик, Ленина, 41",5.172143e+06,1196,3877,"(проспект Ленина, 41/90, Нальчик, Кабардино-Ба..."
6,108,"Ставрополь, 50 лет ВЛКСМ, 16д",4.601245e+06,1251,4123,"(улица 50 лет ВЛКСМ, 16И, Ставрополь, Россия, ..."
7,110,"Анапа, Крымская/Красноармейская",7.631245e+06,2151,4973,"(Крымская улица/Красноармейская улица, Анапа, ..."
8,117,"Курган, Куйбышева, 105",5.714414e+06,1503,5323,"(улица Куйбышева, 105, Курган, Россия, (55.438..."
9,1173,"Омск, Лобкова, 4",2.033405e+06,588,3000,"(улица Лобкова, 4, Омск, Россия, (54.938943, 7..."


In [4]:
import pandas as pd
geo_df = pd.read_excel(r'C:\Users\Dotsenko.Semen\Downloads\словарик_города.xlsx')

numbers = [str(a) for a in list(range(0,10))]
geo_df['district'] = geo_df.full_address.apply(lambda x: x.split(',')[-2] if 
                          any([num in numbers for num in list(x.split(',')[-1])]) 
                          else x.split(',')[-1])

geo_df['region'] = geo_df.full_address.apply(lambda x: x.split(',')[-3] if 
                          any([num in numbers for num in list(x.split(',')[-1])]) 
                          else x.split(',')[-2])

geo_df.to_excel(r'C:\Users\Dotsenko.Semen\Downloads\словарик_города.xlsx')

In [6]:
geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 8 columns):
Unnamed: 0             363 non-null int64
Unnamed: 0.1           363 non-null int64
city                   363 non-null object
full_address           363 non-null object
coord                  363 non-null object
full_address_yandex    363 non-null object
district               363 non-null object
region                 363 non-null object
dtypes: int64(2), object(6)
memory usage: 22.8+ KB


In [1]:
import pandas as pd
def latitude(string):
    return float(string[1:-1].split(",")[0])

def longitude(string):
    return float(string[1:-1].split(",")[1])

In [2]:
geo_df = pd.read_excel(r'C:\Users\Dotsenko.Semen\Downloads\словарик_города.xlsx')
geo_df['latitude'] = geo_df.coord.apply(latitude)
geo_df['longitude'] = geo_df.coord.apply(longitude)

In [3]:
df = geo_df[['latitude','longitude','city']]

In [9]:
from keplergl import KeplerGl
map_1 = KeplerGl()
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl()

In [10]:
map_1.add_data(data=df, name='far')